## Importing Dataset for Tennis Court Keypoints Detection

## Creating Torch Dataset

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np
import os

device = torch.device("mps")

In [28]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

    

In [34]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [30]:
id = train_loader.dataset.data[0]['id']
os.path.exists(f"{id}.png")
print(id)

PuXlxKdUIes_2450


## Implementation of the Model

In [14]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [31]:
model = models.resnet50(pretrained=True) # pretrained model for image classification 
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)  # 14 keypoints with x and y components

model = model.to(device) 

/Users/shounakacharya/Documents/ML_Projects/Tennis_YOLO/tenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shounakacharya/Documents/ML_Projects/Tennis_YOLO/tenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Training/Finetuning the Model

In [32]:
criterion = torch.nn.MSELoss() # loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # optimizer


In [33]:
epochs = 20
for epoch in range(epochs):
    for i, (img,kps) in enumerate(train_loader):

        imgs = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i%100 == 0:
            print(f"Epoch: {epoch+1}/{epochs}, iteration: {i+1}/{len(train_loader)},  Loss: {loss.item()}")


torch.save(model.state_dict(), "keypoints_model.pth") # saving the model


Epoch: 1/20, iteration: 1/829,  Loss: 14366.12109375
Epoch: 1/20, iteration: 101/829,  Loss: 11119.4755859375
Epoch: 1/20, iteration: 201/829,  Loss: 7806.4189453125
Epoch: 1/20, iteration: 301/829,  Loss: 5172.91796875
Epoch: 1/20, iteration: 401/829,  Loss: 3739.484375
Epoch: 1/20, iteration: 501/829,  Loss: 1644.852783203125
Epoch: 1/20, iteration: 601/829,  Loss: 1089.3946533203125
Epoch: 1/20, iteration: 701/829,  Loss: 629.6969604492188
Epoch: 1/20, iteration: 801/829,  Loss: 336.68701171875
Epoch: 2/20, iteration: 1/829,  Loss: 206.34263610839844
Epoch: 2/20, iteration: 101/829,  Loss: 290.23388671875
Epoch: 2/20, iteration: 201/829,  Loss: 57.253631591796875
Epoch: 2/20, iteration: 301/829,  Loss: 69.0167236328125
Epoch: 2/20, iteration: 401/829,  Loss: 79.76292419433594
Epoch: 2/20, iteration: 501/829,  Loss: 29.315753936767578
Epoch: 2/20, iteration: 601/829,  Loss: 30.343725204467773
Epoch: 2/20, iteration: 701/829,  Loss: 33.20823287963867
Epoch: 2/20, iteration: 801/829,  